In [11]:
import pandas as pd
import numpy as np
import statsmodels as sm
from statsmodels.tsa.statespace import sarimax
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import time

from typing import List, Dict
from gretel_client import configure_session

In [12]:
# Specify your Gretel API key

pd.set_option("max_colwidth", None)

configure_session(api_key='grtu9cd34f3901cfac9e13f08002d42705fcbb214dca7a7731179c6a318cb2af2e0a', cache="yes", validate=True)


Using endpoint https://api.gretel.cloud
Logged in as hoang.nd@orai.io ✅


In [13]:
# Load timeseries example to a dataframe

data_source = "./AAPL Historical Data.csv"
original_df = pd.read_csv(data_source)
original_df.to_csv("original.csv", index=False)
original_df


,Date,Price
0,11/01/2013,19.86
1,12/01/2013,20.04
2,01/01/2014,17.88
3,02/01/2014,18.79
4,03/01/2014,19.17
...,...,...
94,09/01/2021,141.50
95,10/01/2021,149.80
96,11/01/2021,165.30
97,12/01/2021,177.57


In [14]:
# Gretel Transforms Configuration
config = """
schema_version: "1.0"
models:
    - transforms:
        data_source: "__tmp__"
        policies:
            - name: shiftnumbers
              rules:
                - name: shiftnumbers
                  conditions:
                    field_name:
                        - Price
                  transforms:
                    - type: numbershift
                      attrs:
                        min: 1
                        max: 100
                        field_name:
                            - Date
              
"""


In [15]:
# De-identify the original dataset using the policy above
import yaml

from gretel_client.projects import create_or_get_unique_project
from gretel_client.helpers import poll

# Create a project and model configuration.
project = create_or_get_unique_project(name="numbershift-transform")

model = project.create_model_obj(
    model_config=yaml.safe_load(config), data_source=data_source
)

# Upload the training data.  Train the model.
model.submit_cloud()
poll(model)

record_handler = model.create_record_handler_obj(data_source=data_source)
record_handler.submit_cloud()
poll(record_handler)

deid_df = pd.read_csv(record_handler.get_artifact_link("data"), compression="gzip")


INFO: Starting poller


{
    "uid": "633fffa938fdbfad39527813",
    "guid": "model_2FnrHZGornojiN5ZY77nhvDkIKl",
    "model_name": "daffy-sweet-salamander",
    "runner_mode": "cloud",
    "user_id": "633ff67bbff621779f449273",
    "user_guid": "user_2FnmWFhqEDJhOMwCmsPcFDdNX3q",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "633ffe8e44a4f15b630a3527",
    "project_guid": "proj_2FnqhzkG3XtUbB0nSxJQHK6Y0fU",
    "status_history": {
        "created": "2022-10-07T10:30:01.475908Z"
    },
    "last_modified": "2022-10-07T10:30:01.601079Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/transforms@sha256:c9b59a72bc0f0d73111d7bc7e52a9c5b8cb6eb44734caa23dc02b1672fa7fc51",
    "container_image_version": "2.10.0",
    "model_type": "transform",
    "config": {
        "schema_vers

INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2022-10-07T10:30:12.387648Z  Starting transforms model training
2022-10-07T10:30:12.388437Z  Loading training data
2022-10-07T10:30:12.641450Z  Using up to 10000 records for training
2022-10-07T10:30:12.641861Z  Training data loaded
2022-10-07T10:30:12.642243Z  Beginning transforms model training
2022-10-07T10:30:15.491884Z  Saving model archive
2022-10-07T10:30:15.493615Z  Saving training report
2022-10-07T10:30:15.494849Z  Generating data preview with up to 100 records
2022-10-07T10:30:15.521707Z  Done generating data preview
2022-10-07T10:30:15.522263Z  Uploading artifacts to Gretel Cloud
2022-10-07T10:30:16.330474Z  Model creation complete!
INFO: Starting poller


{
    "uid": "633fffc3945178bc06f94503",
    "guid": "model_run_2FnrKqG8ynkb3HK9ET396XqKkWK",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "633ff67bbff621779f449273",
    "user_guid": "user_2FnmWFhqEDJhOMwCmsPcFDdNX3q",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "633ffe8e44a4f15b630a3527",
    "project_guid": "proj_2FnqhzkG3XtUbB0nSxJQHK6Y0fU",
    "status_history": {
        "created": "2022-10-07T10:30:27.100000Z"
    },
    "last_modified": "2022-10-07T10:30:27.201000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/transforms@sha256:c9b59a72bc0f0d73111d7bc7e52a9c5b8cb6eb44734caa23dc02b1672fa7fc51",
    "container_image_version": "2.10.0",
    "model_id": "633fffa938fdbfad39527813",
    "model_guid": "model_2FnrHZGornojiN

INFO: Status is pending. A Gretel Cloud worker is being allocated to begin transforming records.
INFO: Status is active. A worker has started!
2022-10-07T10:30:37.684961Z  Downloading model to worker
2022-10-07T10:30:37.918374Z  Loading model
2022-10-07T10:30:37.981013Z  Loading dataset to transform
2022-10-07T10:30:38.299504Z  Starting transform
2022-10-07T10:30:38.321716Z  Transform in progress
{
    "records_transformed": 99,
    "new_records_transformed": 99
}
2022-10-07T10:30:38.322148Z  Dataset transform complete
2022-10-07T10:30:38.329605Z  Uploading data to Gretel Cloud
2022-10-07T10:30:38.542755Z  Uploading run report to Gretel Cloud


In [16]:
# View the transformation report

import json
from smart_open import open

report = json.loads(open(model.get_artifact_link("report_json")).read())
pd.DataFrame(report["metadata"]["fields"])


,name,count,approx_distinct_count,missing_count,labels,attributes,entities,types
0,Date,99,99,0,[datetime],[],"[{'label': 'datetime', 'count': 99, 'approx_distinct_count': 99, 'f_ratio': 1.0, 'sources': ['gretel/datetime']}]","[{'type': 'string', 'count': 99}]"
1,Price,99,97,0,[],[],[],"[{'type': 'number', 'count': 99}]"


In [17]:
# Here we sort and remove "net_amt" as it's a derived column,
# We will add back in after the data is synthesized
train_df = deid_df.copy()

train_df.sort_values("Date", inplace=True)
train_cols = list(train_df.columns)
train_df = train_df.filter(train_cols)

# Here we noticed that some number have extremely long precision,
# so we round the data
train_df = train_df.round(1)
train_df.to_csv("train.csv", index=False)
train_df


,Date,Price
2,01/01/2014,35.9
14,01/01/2015,50.3
26,01/01/2016,60.3
38,01/01/2017,84.3
50,01/01/2018,89.9
...,...,...
49,12/01/2017,127.3
61,12/01/2018,113.4
73,12/01/2019,110.4
85,12/01/2020,136.7


In [19]:
from gretel_client.projects.models import read_model_config

# Create a project and model configuration.
project = create_or_get_unique_project(name="ts-5544-regular-seed")

# Pull down the default synthetic config.  We will modify it slightly.
config = read_model_config("synthetics/default")

# Set up the seed fields
seed_fields = ["Date"]

task = {
    "type": "seed",
    "attrs": {
        "fields": seed_fields,
    },
}

# Fine tune model parameters. These are the parameters we found to work best.  This is "Run 20" in the document
config["models"][0]["synthetics"]["task"] = task

config["models"][0]["synthetics"]["params"]["vocab_size"] = 20
config["models"][0]["synthetics"]["params"]["learning_rate"] = 0.005
config["models"][0]["synthetics"]["params"]["epochs"] = 100
config["models"][0]["synthetics"]["params"]["gen_temp"] = 0.8
config["models"][0]["synthetics"]["params"]["reset_states"] = True
config["models"][0]["synthetics"]["params"]["dropout_rate"] = 0.5
config["models"][0]["synthetics"]["params"]["gen_temp"] = 0.8
config["models"][0]["synthetics"]["params"]["early_stopping"] = True
config["models"][0]["synthetics"]["privacy_filters"]["similarity"] = None
config["models"][0]["synthetics"]["privacy_filters"]["outliers"] = None
config["models"][0]["synthetics"]["generate"]["num_records"] = train_df.shape[0]

# Get a csv to work with, just dump out the train_df.
deid_df.to_csv("train.csv", index=False)

# Initiate a new model with the chosen config
model = project.create_model_obj(model_config=config, data_source="train.csv")

# Upload the training data.  Train the model.
model.submit_cloud()
poll(model)

synthetic = pd.read_csv(model.get_artifact_link("data_preview"), compression="gzip")
synthetic


INFO: Starting poller


{
    "uid": "6340035bec698b6d80724892",
    "guid": "model_2FntCS2DRyKiKERlDMfjWRPJLle",
    "model_name": "default-config",
    "runner_mode": "cloud",
    "user_id": "633ff67bbff621779f449273",
    "user_guid": "user_2FnmWFhqEDJhOMwCmsPcFDdNX3q",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "633fff1f44a4f15b630a3529",
    "project_guid": "proj_2Fnr0BIix7lRhD3Z2BF0BFNT0b7",
    "status_history": {
        "created": "2022-10-07T10:45:47.156562Z"
    },
    "last_modified": "2022-10-07T10:45:47.272155Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/synthetics@sha256:3339cda15d1ac57a84a12275267fe2f59cf27b43e00b916e8f4f154b5e8398a1",
    "container_image_version": "2.10.0",
    "model_type": "synthetics",
    "config": {
        "schema_version": "

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2022-10-07T10:46:17.036954Z  Starting synthetic model training
2022-10-07T10:46:17.039876Z  Loading training data
2022-10-07T10:46:17.145574Z  Running pre-flight data checks on input data
2022-10-07T10:46:17.183355Z  We found 1 warning. If your model fails, try modifying the fields or adjusting your configuration settings.

	1 field with high floating point precision: 'Price' has floats with precision greater than 6 digits. Try reducing precision to 4 digits.
2022-10-07T10:46:17.199856Z  Training data loaded
{
    "record_count": 99,
    "field_count": 2,
    "upsample_count": 9901
}
2022-10-07T10:46:19.865464Z  Creating semantic validators and preparing training data
2022-10-07T10:46:20.805765Z  Beginning ML model training
2022-10-07T10:46:20.806204Z  Running training on 1 b

,Date,Price
0,11/01/2013,57.86
1,12/01/2013,102.04
2,01/01/2014,35.88
3,02/01/2014,48.79
4,03/01/2014,87.17
...,...,...
94,09/01/2021,142.50
95,10/01/2021,170.80
96,11/01/2021,253.30
97,12/01/2021,254.57


In [20]:
import matplotlib
import matplotlib.pyplot as plt


data = pd.read_csv('./original.csv')


KeyError: "None of [Index(['credit_amt_synthetic', 'credit_amt_original'], dtype='object')] are in the [columns]"

<Figure size 640x480 with 0 Axes>